### Objective
- Estimate sequence-level RL using frequency data only

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pickle
from tqdm import tqdm
!pip install camel_tools
from camel_tools.tokenizers.word import simple_word_tokenize


Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.6/426.6 kB 33.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 77.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6

In [ ]:
import pandas as pd

In [ ]:
with open('/content/drive/My Drive/capstone_data/bashar_data/capstone_data/splits/freq_token_db.pkl', 'rb') as f:
  words_and_levels = pickle.load(f)

base_path = '/content/drive/My Drive/capstone_data/bashar_data/capstone_data/splits/'

dev_levelled = pd.read_csv(base_path + 'dev_levelled.csv')

train_levelled = pd.read_csv(base_path + 'train_levelled.csv')

test_levelled = pd.read_csv(base_path + 'test_levelled.csv')

dev_levelled = dev_levelled[dev_levelled.apply(lambda x: type(x['text']) == str, axis = 1)]
train_levelled = train_levelled[train_levelled.apply(lambda x: type(x['text']) == str, axis = 1)]
test_levelled = test_levelled[test_levelled.apply(lambda x: type(x['text']) == str, axis = 1)]

frag_train = pd.read_csv('/content/drive/My Drive/capstone_data/bashar_data/capstone_data/splits/all_train_aligned.csv')
frag_dev = pd.read_csv('/content/drive/My Drive/capstone_data/bashar_data/capstone_data/splits/all_dev_aligned.csv')
frag_test = pd.read_csv('/content/drive/My Drive/capstone_data/bashar_data/capstone_data/splits/all_test_aligned.csv')


frag_train = frag_train[frag_train.apply(lambda x: type(x['0']) == str, axis = 1)]
frag_dev = frag_dev[frag_dev.apply(lambda x: type(x['0']) == str, axis = 1)]
frag_test = frag_test[frag_test.apply(lambda x: type(x['0']) == str, axis = 1)]


In [ ]:
# binning datasets

with open('/content/drive/My Drive/capstone_data/disambig_db/all_levels_freq_binning.pkl', 'rb') as f:
  words_and_levels_freq_binning = pickle.load(f)

with open('/content/drive/My Drive/capstone_data/disambig_db/all_levels_freq_binning_cumulative.pkl', 'rb') as f:
  words_and_levels_freq_binning_cum = pickle.load(f)

In [ ]:
words_and_levels['-']

3

In [ ]:
def highest_aggregation(levels) -> int:
  level = max(levels)
  if level < 3:
    return 3
  else:
    return round(level)

def get_rl(token, model, oov_level = 0):
    try:
        return model[token]
    except:
        return oov_level

def freq_inference_pipeline(fragment, model, aggregation, v = False):
  tokens = simple_word_tokenize(fragment)
  levels = [get_rl(token, model, 0) for token in tokens]
  if v:
    print(tokens)
    print(levels)
  levels = [a for a in levels if a != 0]
  if len(levels) == 0:
    return 3
  return aggregation(levels)

In [ ]:
results_freq = [freq_inference_pipeline(a, words_and_levels, highest_aggregation) for a in test_levelled['text']]


In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(test_levelled['level'], results_freq))

              precision    recall  f1-score   support

           3       0.86      0.24      0.37      3154
           4       0.25      0.25      0.25      1268
           5       0.21      0.83      0.34       777

    accuracy                           0.33      5199
   macro avg       0.44      0.44      0.32      5199
weighted avg       0.61      0.33      0.34      5199



In [ ]:
results_binning_1 = []

for w_set in words_and_levels_freq_binning.values():
  res = [freq_inference_pipeline(a, w_set, highest_aggregation) for a in test_levelled['text']]
  results_binning_1.append(classification_report(test_levelled['level'], res, output_dict = True))



In [ ]:
results_binning_2 = []

for w_set, _ in words_and_levels_freq_binning_cum.values():
  res = [freq_inference_pipeline(a, w_set, highest_aggregation) for a in test_levelled['text']]
  results_binning_2.append(classification_report(test_levelled['level'], res, output_dict = True))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

In [ ]:
import numpy as np

In [ ]:
arr = np.concatenate([[results_binning_1[0][x]['f1-score'],
            results_binning_1[0][x]['precision'],
            results_binning_1[0][x]['recall'],] for x in {'3', '4', '5'}])
np.append(arr, results_binning_1[0]['accuracy'])

array([0.75143006, 0.63684442, 0.91629677, 0.28593041, 0.34678899,
       0.24324324, 0.03612717, 0.21551724, 0.01971609, 0.59703789])

In [2]:
def class_report_to_csv(report_arr):
  reports = []

  for report in report_arr:
    arr = np.concatenate([[report[x]['f1-score'],
            report[x]['precision'],
            report[x]['recall'],] for x in {'3', '4', '5'}])
    arr = np.append(arr, report['accuracy'])
    reports.append(arr)

  return pd.DataFrame(reports)

In [ ]:
class_report_to_csv(results_binning_1).to_csv(
          '/content/drive/My Drive/capstone_data/bashar_data/capstone_data/splits/freq_binning_equal_results.csv')

In [ ]:
class_report_to_csv(results_binning_2).to_csv(
          '/content/drive/My Drive/capstone_data/bashar_data/capstone_data/splits/freq_binning_cum_results.csv')

In [ ]:
words_and_levels_freq_binning_cum.keys()

dict_keys([10000, 5000, 2000, 1000, 500, 200, 100])

In [ ]:
words_and_levels_freq_binning.keys()

dict_keys(['2000', '5000', '10000', '20000', '50000', '100000', '2000_3', '5000_3', '10000_3', '20000_3', '50000_3', '100000_3'])